In [2]:
class Process:
    def __init__(self, pid, arrival_time, burst_time, priority=None):
        self.pid = pid  # Process ID
        self.arrival_time = arrival_time  # Time at which the process arrives in the scheduling queue
        self.burst_time = burst_time  # Total time the process requires on the CPU
        self.priority = priority  # Priority of the process (lower number = higher priority)
        self.remaining_burst_time = burst_time  # To track the remaining burst time during scheduling
        self.start_time = None  # To record when the process starts execution
        self.completion_time = None  # To record when the process completes execution

    def __repr__(self):
        return (f"Process(pid={self.pid}, arrival_time={self.arrival_time}, "
                f"burst_time={self.burst_time}, priority={self.priority})")


In [3]:
import random

def generate_random_processes(num_processes, arrival_time_range, burst_time_range, priority_range=None):
    processes = []
    for pid in range(1, num_processes + 1):
        arrival_time = random.randint(*arrival_time_range)
        burst_time = random.randint(*burst_time_range)
        priority = random.randint(*priority_range) if priority_range else None
        processes.append(Process(pid, arrival_time, burst_time, priority))
    return processes

def read_processes_from_file(filename):
    processes = []
    with open(filename, 'r') as file:
        for line in file:
            parts = line.strip().split()
            pid = int(parts[0])
            arrival_time = int(parts[1])
            burst_time = int(parts[2])
            priority = int(parts[3]) if len(parts) > 3 else None
            processes.append(Process(pid, arrival_time, burst_time, priority))
    return processes



In [4]:
def fcfs_scheduling(processes):
    # Sort processes by arrival time
    processes.sort(key=lambda x: x.arrival_time)

    # Initialize variables
    current_time = 0
    total_waiting_time = 0
    total_turnaround_time = 0

    print("Process Execution Order:")
    for process in processes:
        if current_time < process.arrival_time:
            current_time = process.arrival_time  # If CPU is idle, wait until the process arrives

        process.start_time = current_time
        process.completion_time = process.start_time + process.burst_time
        process.waiting_time = process.start_time - process.arrival_time
        process.turnaround_time = process.completion_time - process.arrival_time

        # Update current time to the completion time of the current process
        current_time = process.completion_time

        # For results display and analysis
        total_waiting_time += process.waiting_time
        total_turnaround_time += process.turnaround_time

        print(f"{process}")

    # Calculating average times
    average_waiting_time = total_waiting_time / len(processes)
    average_turnaround_time = total_turnaround_time / len(processes)
    
    print(f"\nAverage Waiting Time: {average_waiting_time:.2f}")
    print(f"Average Turnaround Time: {average_turnaround_time:.2f}")

    return processes  # Returning the list of processes might be useful for further analysis or visualization


In [5]:
def sjf_scheduling(processes):
    # Sort processes by arrival time initially to simulate arrival
    processes.sort(key=lambda x: x.arrival_time)
    
    current_time = 0
    completed_processes = []
    ready_queue = []
    total_waiting_time = 0
    total_turnaround_time = 0

    while processes or ready_queue:
        # Add processes that have arrived by the current time to the ready queue
        while processes and processes[0].arrival_time <= current_time:
            ready_queue.append(processes.pop(0))
        
        if not ready_queue:
            # If no process is ready, advance time to the next process arrival
            current_time = processes[0].arrival_time
            continue
        
        # Select the process with the shortest burst time
        ready_queue.sort(key=lambda x: x.burst_time)
        current_process = ready_queue.pop(0)
        
        # Calculate timing details
        current_process.start_time = current_time
        current_process.completion_time = current_process.start_time + current_process.burst_time
        current_process.waiting_time = current_process.start_time - current_process.arrival_time
        current_process.turnaround_time = current_process.completion_time - current_process.arrival_time

        # Update the current time
        current_time = current_process.completion_time
        
        # Keep track of completed processes
        completed_processes.append(current_process)
        
        # Calculate totals for average calculations
        total_waiting_time += current_process.waiting_time
        total_turnaround_time += current_process.turnaround_time

    average_waiting_time = total_waiting_time / len(completed_processes)
    average_turnaround_time = total_turnaround_time / len(completed_processes)
    
    print("Process Execution Order:")
    for process in completed_processes:
        print(f"{process}")
    print(f"\nAverage Waiting Time: {average_waiting_time:.2f}")
    print(f"Average Turnaround Time: {average_turnaround_time:.2f}")

    return completed_processes


In [6]:
def priority_scheduling(processes):
    # Sort processes by arrival time to simulate arrival order
    processes.sort(key=lambda x: (x.arrival_time, x.priority))
    
    current_time = 0
    completed_processes = []
    ready_queue = []
    total_waiting_time = 0
    total_turnaround_time = 0

    while processes or ready_queue:
        # Add processes that have arrived by the current time to the ready queue
        while processes and processes[0].arrival_time <= current_time:
            ready_queue.append(processes.pop(0))

        if not ready_queue:
            # If no process is ready, advance time to the next process arrival
            current_time = processes[0].arrival_time
            continue
        
        # Select the process with the highest priority (lowest numerical value)
        ready_queue.sort(key=lambda x: x.priority)
        current_process = ready_queue.pop(0)

        # Calculate timing details
        current_process.start_time = current_time
        current_process.completion_time = current_process.start_time + current_process.burst_time
        current_process.waiting_time = current_process.start_time - current_process.arrival_time
        current_process.turnaround_time = current_process.completion_time - current_process.arrival_time

        # Update the current time
        current_time = current_process.completion_time
        
        # Keep track of completed processes
        completed_processes.append(current_process)
        
        # Calculate totals for average calculations
        total_waiting_time += current_process.waiting_time
        total_turnaround_time += current_process.turnaround_time

    average_waiting_time = total_waiting_time / len(completed_processes)
    average_turnaround_time = total_turnaround_time / len(completed_processes)
    
    print("Process Execution Order:")
    for process in completed_processes:
        print(f"{process}")
    print(f"\nAverage Waiting Time: {average_waiting_time:.2f}")
    print(f"Average Turnaround Time: {average_turnaround_time:.2f}")

    return completed_processes


In [7]:
def round_robin_scheduling(processes, time_quantum):
    # Sort processes by arrival time
    processes.sort(key=lambda x: x.arrival_time)
    
    current_time = 0
    queue = []
    completed_processes = []
    total_waiting_time = 0
    total_turnaround_time = 0

    # Initialize the queue with the first process(es) that arrive at the beginning
    while processes and processes[0].arrival_time <= current_time:
        queue.append(processes.pop(0))

    while queue:
        current_process = queue.pop(0)
        # Check if this is the first time the process is running
        if current_process.start_time is None:
            current_process.start_time = current_time

        # Calculate the execution time
        execution_time = min(current_process.remaining_burst_time, time_quantum)
        current_process.remaining_burst_time -= execution_time
        current_time += execution_time

        # Check if process is finished
        if current_process.remaining_burst_time == 0:
            current_process.completion_time = current_time
            current_process.turnaround_time = current_process.completion_time - current_process.arrival_time
            current_process.waiting_time = current_process.turnaround_time - current_process.burst_time
            completed_processes.append(current_process)
        else:
            # Reinsert the process at the end of the queue
            queue.append(current_process)

        # Add new processes to the queue that have arrived while the current process was running
        while processes and processes[0].arrival_time <= current_time:
            queue.append(processes.pop(0))

    # Calculate average metrics after all processes are completed
    for process in completed_processes:
        total_waiting_time += process.waiting_time
        total_turnaround_time += process.turnaround_time

    average_waiting_time = total_waiting_time / len(completed_processes)
    average_turnaround_time = total_turnaround_time / len(completed_processes)

    print("Process Execution Order:")
    for process in completed_processes:
        print(f"{process}")
    print(f"\nAverage Waiting Time: {average_waiting_time:.2f}")
    print(f"Average Turnaround Time: {average_turnaround_time:.2f}")

    return completed_processes


In [8]:
from collections import defaultdict, deque

def priority_round_robin_scheduling(processes, time_quantum):
    # Organize processes into priority queues
    priority_queues = defaultdict(deque)
    for process in sorted(processes, key=lambda x: (x.arrival_time, x.priority)):
        priority_queues[process.priority].append(process)
    
    current_time = 0
    completed_processes = []
    total_waiting_time = 0
    total_turnaround_time = 0

    # Process each priority level
    for priority in sorted(priority_queues):
        queue = priority_queues[priority]
        while queue:
            current_process = queue.popleft()
            if current_process.start_time is None:
                current_process.start_time = current_time
            
            # Calculate the execution time
            execution_time = min(current_process.remaining_burst_time, time_quantum)
            current_process.remaining_burst_time -= execution_time
            current_time += execution_time

            # Check if process is finished
            if current_process.remaining_burst_time == 0:
                current_process.completion_time = current_time
                current_process.turnaround_time = current_process.completion_time - current_process.arrival_time
                current_process.waiting_time = current_process.turnaround_time - current_process.burst_time
                completed_processes.append(current_process)
            else:
                # Process not finished, requeue it
                queue.append(current_process)

    # Calculate average metrics
    for process in completed_processes:
        total_waiting_time += process.waiting_time
        total_turnaround_time += process.turnaround_time

    average_waiting_time = total_waiting_time / len(completed_processes)
    average_turnaround_time = total_turnaround_time / len(completed_processes)

    print("Process Execution Order:")
    for process in completed_processes:
        print(f"{process}")
    print(f"\nAverage Waiting Time: {average_waiting_time:.2f}")
    print(f"Average Turnaround Time: {average_turnaround_time:.2f}")

    return completed_processes


In [9]:
def calculate_average_waiting_time(processes):
    total_waiting_time = sum(process.waiting_time for process in processes)
    return total_waiting_time / len(processes) if processes else 0

def calculate_average_turnaround_time(processes):
    total_turnaround_time = sum(process.turnaround_time for process in processes)
    return total_turnaround_time / len(processes) if processes else 0

def calculate_cpu_utilization(processes, total_simulation_time):
    busy_time = sum(process.burst_time for process in processes)
    return (busy_time / total_simulation_time) * 100 if total_simulation_time > 0 else 0


In [13]:
import matplotlib.pyplot as plt

def plot_waiting_and_turnaround_times(processes):
    pids = [process.pid for process in processes]
    waiting_times = [process.waiting_time for process in processes]
    turnaround_times = [process.turnaround_time for process in processes]

    fig, ax = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
    ax[0].bar(pids, waiting_times, color='skyblue')
    ax[0].set_title('Waiting Time for Each Process')
    ax[0].set_ylabel('Waiting Time')

    ax[1].bar(pids, turnaround_times, color='lightgreen')
    ax[1].set_title('Turnaround Time for Each Process')
    ax[1].set_ylabel('Turnaround Time')
    ax[1].set_xlabel('Process ID')

    plt.tight_layout()
    plt.show()

def plot_cpu_utilization(timeline, total_simulation_time):
    time_steps = list(range(total_simulation_time))
    utilization = [0 if x == 'idle' else 1 for x in timeline]

    plt.figure(figsize=(12, 4))
    plt.fill_between(time_steps, utilization, step='post', color='gray', alpha=0.5)
    plt.title('CPU Utilization Over Time')
    plt.xlabel('Time')
    plt.ylabel('CPU Utilization (0=Idle, 1=Busy)')
    plt.yticks([0, 1], ['Idle', 'Busy'])
    plt.show()

def visualize_scheduling_2(processes, total_simulation_time):
    # Prepare timeline data for CPU utilization plot
    timeline = ['idle'] * total_simulation_time
    for process in processes:
        for t in range(process.start_time, process.completion_time):
            timeline[t] = f'P{process.pid}'

    # Display the timeline and detailed view
    display_timeline(processes, total_simulation_time)
    # plot_waiting_and_turnaround_times(processes)
    # plot_cpu_utilization(timeline, total_simulation_time)

    # Calculate and display performance metrics
    avg_waiting_time = calculate_average_waiting_time(processes)
    avg_turnaround_time = calculate_average_turnaround_time(processes)
    cpu_utilization = calculate_cpu_utilization(processes, total_simulation_time)

    print("\nPerformance Metrics:")
    print(f"Average Waiting Time: {avg_waiting_time:.2f} units")
    print(f"Average Turnaround Time: {avg_turnaround_time:.2f} units")
    print(f"CPU Utilization: {cpu_utilization:.2f}%")

# Example display_timeline function for completeness
def display_timeline(processes, total_simulation_time):
    print("Timeline:")
    timeline = ['idle'] * total_simulation_time
    for process in processes:
        for t in range(process.start_time, process.completion_time):
            timeline[t] = f'P{process.pid}'

    print(' '.join(timeline))
    print("\nDetailed Process View:")
    print(f"{'PID':>3} {'Start':>6} {'End':>4} {'Duration':>8} {'Wait':>5} {'Turnaround':>11}")
    for process in processes:
        print(f"{process.pid:>3} {process.start_time:>6} {process.completion_time:>4} "
              f"{process.burst_time:>8} {process.waiting_time:>5} {process.turnaround_time:>11}")


In [15]:
def test_case_1():
    processes = [
        Process(pid=1, arrival_time=0, burst_time=10, priority=1),
        Process(pid=2, arrival_time=0, burst_time=5, priority=2),
        Process(pid=3, arrival_time=0, burst_time=2, priority=3)
    ]

    # FCFS*
    print("FCFS Results:")
    fcfs_result = fcfs_scheduling(processes.copy())
    
    # visualize_scheduling_2(fcfs_result, max(p.completion_time for p in fcfs_result))

    # SJF
    print("SJF Results:")
    sjf_result = sjf_scheduling(processes.copy())
    
    # visualize_scheduling_2(sjf_result, max(p.completion_time for p in sjf_result))

    # Priority Scheduling
    print("Priority Scheduling Results:")
    priority_result = priority_scheduling(processes.copy())
    
    # visualize_scheduling_2(priority_result, max(p.completion_time for p in priority_result))

if __name__ == "__main__":
    test_case_1()


FCFS Results:
Process Execution Order:
Process(pid=1, arrival_time=0, burst_time=10, priority=1)
Process(pid=2, arrival_time=0, burst_time=5, priority=2)
Process(pid=3, arrival_time=0, burst_time=2, priority=3)

Average Waiting Time: 8.33
Average Turnaround Time: 14.00
SJF Results:
Process Execution Order:
Process(pid=3, arrival_time=0, burst_time=2, priority=3)
Process(pid=2, arrival_time=0, burst_time=5, priority=2)
Process(pid=1, arrival_time=0, burst_time=10, priority=1)

Average Waiting Time: 3.00
Average Turnaround Time: 8.67
Priority Scheduling Results:
Process Execution Order:
Process(pid=1, arrival_time=0, burst_time=10, priority=1)
Process(pid=2, arrival_time=0, burst_time=5, priority=2)
Process(pid=3, arrival_time=0, burst_time=2, priority=3)

Average Waiting Time: 8.33
Average Turnaround Time: 14.00


In [17]:
def test_rr_and_priority_rr():
    processes = [
        Process(pid=1, arrival_time=0, burst_time=10, priority=1),
        Process(pid=2, arrival_time=0, burst_time=5, priority=2),
        Process(pid=3, arrival_time=0, burst_time=2, priority=3)
    ]
    time_quantum = 3

    # Round Robin
    print("Round Robin Results:")
    rr_result = round_robin_scheduling(processes.copy(), time_quantum)
    # visualize_scheduling_2(rr_result, max(p.completion_time for p in rr_result))

    # Priority + Round Robin
    print("Priority + Round Robin Results:")
    pr_rr_result = priority_round_robin_scheduling(processes.copy(), time_quantum)
    # visualize_scheduling_2(pr_rr_result, max(p.completion_time for p in pr_rr_result))

if __name__ == "__main__":
    test_rr_and_priority_rr()


Round Robin Results:
Process Execution Order:
Process(pid=3, arrival_time=0, burst_time=2, priority=3)
Process(pid=2, arrival_time=0, burst_time=5, priority=2)
Process(pid=1, arrival_time=0, burst_time=10, priority=1)

Average Waiting Time: 7.00
Average Turnaround Time: 12.67
Timeline:
P1 P1 P1 P1 P1 P1 P1 P1 P1 P1 P1 P1 P1 P1 P1 P1 P1

Detailed Process View:
PID  Start  End Duration  Wait  Turnaround
  3      6    8        2     6           8
  2      3   13        5     8          13
  1      0   17       10     7          17

Performance Metrics:
Average Waiting Time: 7.00 units
Average Turnaround Time: 12.67 units
CPU Utilization: 100.00%
Priority + Round Robin Results:
Process Execution Order:
Process(pid=1, arrival_time=0, burst_time=10, priority=1)
Process(pid=2, arrival_time=0, burst_time=5, priority=2)
Process(pid=3, arrival_time=0, burst_time=2, priority=3)

Average Waiting Time: -5.67
Average Turnaround Time: 0.00
Timeline:


Detailed Process View:
PID  Start  End Duration  

In [12]:
# def run_test_cases():
#     # Test Case 1: All processes arrive at the same time
#     processes_same_arrival = [
#         Process(pid=1, arrival_time=0, burst_time=10, priority=1),
#         Process(pid=2, arrival_time=0, burst_time=5, priority=2),
#         Process(pid=3, arrival_time=0, burst_time=2, priority=3)
#     ]

#     # Test Case 2: Processes arrive at different times
#     processes_different_arrival = [
#         Process(pid=1, arrival_time=0, burst_time=8, priority=2),
#         Process(pid=2, arrival_time=1, burst_time=4, priority=1),
#         Process(pid=3, arrival_time=2, burst_time=9, priority=3)
#     ]

#     # Execute FCFS on test case 1
#     fcfs_result = fcfs_scheduling(processes_same_arrival.copy())
#     visualize_scheduling_2(fcfs_result, max(p.completion_time for p in fcfs_result))

#     # Execute SJF on test case 1
#     sjf_result = sjf_scheduling(processes_same_arrival.copy())
#     visualize_scheduling_2(sjf_result, max(p.completion_time for p in sjf_result))

#     # Execute Priority Scheduling on test case 2
#     priority_result = priority_scheduling(processes_different_arrival.copy())
#     visualize_scheduling_2(priority_result, max(p.completion_time for p in priority_result))

#     # Execute RR on test case 1 with time quantum = 3
#     rr_result = round_robin_scheduling(processes_same_arrival.copy(), time_quantum=3)
#     visualize_scheduling_2(rr_result, max(p.completion_time for p in rr_result))

#     # Execute Priority + RR on test case 2 with time quantum = 4
#     pr_rr_result = priority_round_robin_scheduling(processes_different_arrival.copy(), time_quantum=4)
#     visualize_scheduling_2(pr_rr_result, max(p.completion_time for p in pr_rr_result))

# if __name__ == "__main__":
#     run_test_cases()


Process Execution Order:
Process(pid=3, arrival_time=0, burst_time=2, priority=3)
Process(pid=2, arrival_time=0, burst_time=5, priority=2)
Process(pid=1, arrival_time=0, burst_time=10, priority=1)

Average Waiting Time: 7.00
Average Turnaround Time: 12.67
Round Robin Results:
Timeline:
P1 P1 P1 P1 P1 P1 P1 P1 P1 P1 P1 P1 P1 P1 P1 P1 P1

Detailed Process View:
PID  Start  End Duration  Wait  Turnaround
  3      6    8        2     6           8
  2      3   13        5     8          13
  1      0   17       10     7          17

Performance Metrics:
Average Waiting Time: 7.00 units
Average Turnaround Time: 12.67 units
CPU Utilization: 100.00%
Process Execution Order:
Process(pid=1, arrival_time=0, burst_time=10, priority=1)
Process(pid=2, arrival_time=0, burst_time=5, priority=2)
Process(pid=3, arrival_time=0, burst_time=2, priority=3)

Average Waiting Time: -5.67
Average Turnaround Time: 0.00
Priority + Round Robin Results:
Timeline:


Detailed Process View:
PID  Start  End Duration  

In [40]:
from collections import deque

def round_robin_scheduling(processes, time_quantum):
    queue = deque(processes)
    current_time = 0
    completed_processes = []

    # Initialize remaining burst time and start time at first use
    for proc in processes:
        proc.remaining_burst_time = proc.burst_time
        proc.start_time = None

    while queue:
        current_process = queue.popleft()
        # Set the initial start time if it's the first time the process is run
        if current_process.start_time is None:
            current_process.start_time = current_time

        # Calculate the execution time
        execution_time = min(current_process.remaining_burst_time, time_quantum)
        current_process.remaining_burst_time -= execution_time
        current_time += execution_time

        # Check if the process is completed
        if current_process.remaining_burst_time > 0:
            queue.append(current_process)  # Requeue the process if not finished
        else:
            # Process completes
            current_process.completion_time = current_time
            current_process.turnaround_time = current_process.completion_time - current_process.arrival_time
            current_process.waiting_time = current_process.turnaround_time - current_process.burst_time
            completed_processes.append(current_process)  # Add to completed processes

    return completed_processes


from collections import defaultdict, deque

def priority_round_robin_scheduling(processes, time_quantum):
    # Organize processes into priority queues
    priority_queues = defaultdict(deque)
    for process in sorted(processes, key=lambda x: (x.priority, x.arrival_time)):
        priority_queues[process.priority].append(process)
    
    current_time = 0
    completed_processes = []

    # Process each priority level sorted by priority
    for priority in sorted(priority_queues.keys()):
        queue = priority_queues[priority]
        while queue:
            current_process = queue.popleft()

            # Calculate the execution start time
            if current_process.start_time is None:
                current_process.start_time = max(current_time, current_process.arrival_time)

            # Run the process for a quantum or until completion
            execution_time = min(current_process.remaining_burst_time, time_quantum)
            current_process.remaining_burst_time -= execution_time
            current_time += execution_time

            if current_process.remaining_burst_time > 0:
                # Requeue the process if not completed
                queue.append(current_process)
            else:
                # Finalize the process if completed
                current_process.completion_time = current_time
                current_process.turnaround_time = current_process.completion_time - current_process.arrival_time
                current_process.waiting_time = current_process.turnaround_time - current_process.burst_time
                completed_processes.append(current_process)

    return completed_processes


In [63]:
def test_round_robin():
    processes = [
        Process(pid=1, arrival_time=0, burst_time=5, priority=1),
        Process(pid=2, arrival_time=1, burst_time=4, priority=1),
        Process(pid=3, arrival_time=2, burst_time=2, priority=1),
        Process(pid=4, arrival_time=3, burst_time=1, priority=1)
    ]
    time_quantum = 2

    # Run Round Robin Scheduling
    rr_result = round_robin_scheduling(processes, time_quantum)
    print("Round Robin Results:")
    for proc in rr_result:
        print(f"PID: {proc.pid}, Waiting Time: {proc.waiting_time}, Turnaround Time: {proc.turnaround_time}")

if __name__ == "__main__":
    test_round_robin()


Round Robin Results:
PID: 3, Waiting Time: 2, Turnaround Time: 4
PID: 4, Waiting Time: 3, Turnaround Time: 4
PID: 2, Waiting Time: 6, Turnaround Time: 10
PID: 1, Waiting Time: 7, Turnaround Time: 12


In [57]:
def test_round_robin():
    processes = [
        Process(pid=1, arrival_time=0, burst_time=5, priority=1),
        Process(pid=2, arrival_time=1, burst_time=4, priority=1),
        Process(pid=3, arrival_time=2, burst_time=1, priority=1),
        Process(pid=4, arrival_time=3, burst_time=2, priority=1),
    ]
    time_quantum = 3

    # Run Round Robin Scheduling
    rr_result = round_robin_scheduling(processes, time_quantum)
    print("Round Robin Results:")
    for proc in rr_result:
        print(f"PID: {proc.pid}, Waiting Time: {proc.waiting_time}, Turnaround Time: {proc.turnaround_time}")

if __name__ == "__main__":
    test_round_robin()


Round Robin Results:
PID: 2, Waiting Time: 2, Turnaround Time: 5
PID: 3, Waiting Time: 4, Turnaround Time: 5
PID: 4, Waiting Time: 4, Turnaround Time: 6
PID: 5, Waiting Time: 5, Turnaround Time: 8
PID: 1, Waiting Time: 9, Turnaround Time: 14


In [59]:
def test_round_robin2():
    processes = [
        Process(pid=1, arrival_time=0, burst_time=10, priority=1),
        Process(pid=2, arrival_time=2, burst_time=15, priority=2),
        Process(pid=3, arrival_time=4, burst_time=20, priority=3),
        Process(pid=4, arrival_time=6, burst_time=5, priority=4),
        Process(pid=5, arrival_time=8, burst_time=15, priority=5),
        Process(pid=6, arrival_time=10, burst_time=10, priority=6)
    ]
    time_quantum = 4

    # Run Round Robin Scheduling
    rr_result = round_robin_scheduling(processes, time_quantum)
    print("Round Robin Results:")
    for proc in rr_result:
        print(f"PID: {proc.pid}, Waiting Time: {proc.waiting_time}, Turnaround Time: {proc.turnaround_time}")

if __name__ == "__main__":
    test_round_robin2()

Round Robin Results:
PID: 4, Waiting Time: 26, Turnaround Time: 31
PID: 1, Waiting Time: 37, Turnaround Time: 47
PID: 6, Waiting Time: 41, Turnaround Time: 51
PID: 2, Waiting Time: 47, Turnaround Time: 62
PID: 5, Waiting Time: 48, Turnaround Time: 63
PID: 3, Waiting Time: 51, Turnaround Time: 71


In [52]:
def test_priority_round_robin():
    processes = [
        Process(pid=1, arrival_time=0, burst_time=10, priority=1),
        Process(pid=2, arrival_time=0, burst_time=5, priority=2),
        Process(pid=3, arrival_time=0, burst_time=2, priority=3)
    ]
    time_quantum = 3

    # Run Priority + Round Robin Scheduling
    pr_rr_result = priority_round_robin_scheduling(processes, time_quantum)
    print("Priority + Round Robin Results:")
    for proc in pr_rr_result:
        print(f"PID: {proc.pid}, Waiting Time: {proc.waiting_time}, Turnaround Time: {proc.turnaround_time}")

if __name__ == "__main__":
    test_priority_round_robin()

Priority + Round Robin Results:
PID: 1, Waiting Time: 0, Turnaround Time: 10
PID: 2, Waiting Time: 10, Turnaround Time: 15
PID: 3, Waiting Time: 15, Turnaround Time: 17
